In [7]:
import pandas as pd
import os
import utils
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings("ignore")

In [8]:
dfMetaData = utils.concat('../data/Google_Maps/metadata-sitios')

Cargando 1.json
Cargando 10.json
Cargando 11.json
Cargando 2.json
Cargando 3.json
Cargando 4.json
Cargando 5.json
Cargando 6.json
Cargando 7.json
Cargando 8.json
Cargando 9.json


In [9]:
dfMetaData.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.215290,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
3,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,None,34.023669,-118.232930,[Fabric store],3.3,6,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Service options': ['In-store shopping'], 'Pa...",Open ⋅ Closes 5PM,"[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
4,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,[Fabric store],4.3,7,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",{'Service options': ['In-store pickup']},Open ⋅ Closes 5PM,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [10]:
# Drop rows with None values
dfMetaDataBanks = dfMetaData[['gmap_id','category']].dropna(subset=['category'])

# Extract the first one or two values from the category
dfMetaDataBanks['category'] = dfMetaDataBanks['category'].apply(lambda x: x[:2] if (x is not None and len(x) >= 2) else [x[0]] if (x is not None and len(x) == 1) else [])

# Explode the DataFrame to have each value in a separate row
dfMetaDataBanks = dfMetaDataBanks.explode('category')

dfMetaDataBanks = dfMetaDataBanks[dfMetaDataBanks['category'] == 'Bank'].drop('category', axis=1)

# Reset index if needed
dfMetaDataBanks.reset_index(drop=True, inplace=True)


In [11]:
dfMetaData = dfMetaData[dfMetaData['gmap_id'].isin(dfMetaDataBanks['gmap_id'].unique())].reset_index(drop=True)
dfMetaData = dfMetaData.drop(columns=['address','description','price','hours','MISC','state','relative_results','category'])

In [12]:
utils.dataType(dfMetaData)

,name,data_type,not_null_%,null_%,not_null,null
0,name,[<class 'str'>],100.0,0.0,23871,0
1,gmap_id,[<class 'str'>],100.0,0.0,23871,0
2,latitude,[<class 'float'>],100.0,0.0,23871,0
3,longitude,[<class 'float'>],100.0,0.0,23871,0
4,avg_rating,[<class 'float'>],100.0,0.0,23871,0
5,num_of_reviews,[<class 'int'>],100.0,0.0,23871,0
6,url,[<class 'str'>],100.0,0.0,23871,0


In [13]:
dfMetaData = dfMetaData.drop_duplicates()

In [14]:
dfMetaData[dfMetaData['gmap_id'].duplicated()]

,name,gmap_id,latitude,longitude,avg_rating,num_of_reviews,url


In [15]:
table = pa.Table.from_pandas(dfMetaData)
pq.write_table(table, '../data/Google_Maps/metadata-sitios/metadata-sitios.snappy.parquet', compression='SNAPPY')

In [16]:
dfMetaData.to_csv('../data/Cloud_Upload/Google_Maps/metadata/metadata.csv', encoding='utf-8', index=False)

In [17]:
table = pa.Table.from_pandas(dfMetaDataBanks)
pq.write_table(table, '../data/Google_Maps/metadata-sitios/banks-id.snappy.parquet', compression='SNAPPY')

In [18]:
# dfMetaData.to_parquet('../data/Google_Maps/metadata-sitios/metadata-sitios.parquet', index = False)

In [36]:
def etlReviews(folderPath, gmap):
    for folderName, subfolders, fileNames in os.walk(folderPath):
        resultDf = pd.DataFrame()
        filePath = ''
        for fileName in fileNames:
            filePath = os.path.join(folderName, fileName)
            df = utils.loadFile(filePath)
            print(f'Cargando {filePath}...')
            if not filePath.endswith('.json'):
                os.remove(filePath)
                filePath = '.json'
            else:
                if not resultDf.empty:
                    resultDf = pd.concat([resultDf, df], axis = 0)
                else:
                    resultDf = df
        if filePath.endswith('.json'):
            resultDf = resultDf.drop(columns=['pics','resp'])

            resultDf = resultDf[resultDf['gmap_id'].isin(gmap['gmap_id'].unique())].reset_index(drop=True)

            if resultDf.duplicated().any():
                print(f'Se eliminaron filas duplicadas en {folderName}')
                resultDf = resultDf.drop_duplicates()

            resultDf['text'] = resultDf['text'].fillna('No comment')
            resultDf['text'] = resultDf['text'].replace('None', 'No comment')
            resultDf['text'] = resultDf['text'].replace('N/A', 'No comment')
            resultDf['name'] = resultDf['name'].replace('None', 'Anonymous')

            # ETL
            resultDf['time'] = pd.to_datetime(resultDf['time'], unit='ms')
            resultDf['date'] = pd.to_datetime(resultDf['time']).dt.date
            resultDf['hour'] = pd.to_datetime(resultDf['time']).dt.hour
            resultDf['time'] = pd.to_datetime(resultDf['time']).dt.time

            state = (folderName.split('/')[-1]).replace('review-', '')

            resultDf['state'] = state

            table = pa.Table.from_pandas(resultDf)
            pq.write_table(table, f'../data/Google_Maps/reviews-estados-snappy/{state}.snappy.parquet', compression='SNAPPY')
            # resultDf.to_parquet(f'{folderName}/{state}.parquet', index = False)
    return print('ETL finalizado')

In [20]:
table = pq.read_table('../data/Google_Maps/metadata-sitios/banks-id.snappy.parquet')
gmapDf = table.to_pandas()

In [37]:
etlReviews('../data/Google_Maps/reviews-estados/', gmapDf)

Cargando ../data/Google_Maps/reviews-estados/review-Alabama\1.json...
Cargando ../data/Google_Maps/reviews-estados/review-Alabama\10.json...
Cargando ../data/Google_Maps/reviews-estados/review-Alabama\11.json...
Cargando ../data/Google_Maps/reviews-estados/review-Alabama\12.json...
Cargando ../data/Google_Maps/reviews-estados/review-Alabama\2.json...
Cargando ../data/Google_Maps/reviews-estados/review-Alabama\3.json...
Cargando ../data/Google_Maps/reviews-estados/review-Alabama\4.json...
Cargando ../data/Google_Maps/reviews-estados/review-Alabama\5.json...
Cargando ../data/Google_Maps/reviews-estados/review-Alabama\6.json...
Cargando ../data/Google_Maps/reviews-estados/review-Alabama\7.json...
Cargando ../data/Google_Maps/reviews-estados/review-Alabama\8.json...
Cargando ../data/Google_Maps/reviews-estados/review-Alabama\9.json...
Cargando ../data/Google_Maps/reviews-estados/review-Alaska\1.json...
Cargando ../data/Google_Maps/reviews-estados/review-Alaska\2.json...
Cargando ../data/Go

In [38]:
reviewsDf = utils.concat('../data/Google_Maps/reviews-estados-snappy')

Cargando Alabama.snappy.parquet
Cargando Alaska.snappy.parquet
Cargando Arizona.snappy.parquet
Cargando Arkansas.snappy.parquet
Cargando California.snappy.parquet
Cargando Colorado.snappy.parquet
Cargando Connecticut.snappy.parquet
Cargando Delaware.snappy.parquet
Cargando District_of_Columbia.snappy.parquet
Cargando Florida.snappy.parquet
Cargando Georgia.snappy.parquet
Cargando Hawaii.snappy.parquet
Cargando Idaho.snappy.parquet
Cargando Illinois.snappy.parquet
Cargando Indiana.snappy.parquet
Cargando Iowa.snappy.parquet
Cargando Kansas.snappy.parquet
Cargando Kentucky.snappy.parquet
Cargando Louisiana.snappy.parquet
Cargando Maine.snappy.parquet
Cargando Maryland.snappy.parquet
Cargando Massachusetts.snappy.parquet
Cargando Michigan.snappy.parquet
Cargando Minnesota.snappy.parquet
Cargando Mississippi.snappy.parquet
Cargando Missouri.snappy.parquet
Cargando Montana.snappy.parquet
Cargando Nebraska.snappy.parquet
Cargando Nevada.snappy.parquet
Cargando New_Hampshire.snappy.parquet
Ca

In [39]:
reviewsDf.to_csv('../data/Cloud_Upload/Google_Maps/reviews/reviews.csv', encoding='utf-8', index=False)

In [40]:
table = pa.Table.from_pandas(reviewsDf)
pq.write_table(table, f'../data/Google_Maps/reviews.snappy.parquet', compression='SNAPPY')